### This notebook will contain the loading of the datasets, and tie together the different parts of the project

In [17]:
import numpy as np
import pandas as pd
from DataLoader import DataLoader
from models.CNN import *
from models.ffn import *
import torch
import torch.nn as nn
from models.CointegrationResidualGenerator import CointegrationResidualGenerator
from models.BacktestSharpeEvaluator import BacktestSharpeEvaluator
from PortfolioOptimizer import PortfolioOptimizer
from TimeSeriesDataSplitter import TimeSeriesDataSplitter



## Sharpe ratio

In [5]:
def sharpe_ratio_loss(returns, risk_free_rate=0.0):
    """
    Custom loss function to maximize the Sharpe Ratio.
    Args:
        returns (torch.Tensor): Predicted returns
        risk_free_rate (float): Risk-free rate for Sharpe calculation
    Returns:
        torch.Tensor: Negative Sharpe Ratio (to minimize)
    """
    excess_returns = returns - risk_free_rate
    mean_excess = torch.mean(excess_returns)
    std_excess = torch.std(excess_returns, unbiased=False) + 1e-6  # epsilon for stability
    sharpe_ratio = mean_excess / std_excess
    return -sharpe_ratio

In [15]:

# MAIN FUNCTION
### -------------- PARSING DATA -------------- ###
parser = DataLoader(file_path="../data/european_wholesale_electricity_price_data_daily.csv")

# Get list of countries
print("\n--- List of Countries ---")
all_countries_list = parser.get_country_list()
print(all_countries_list)

# Get daily price matrix for all countries for the entire year 2021
price_matrix = parser.get_price_matrix(
    time_range="2015-01-01,2024-12-31",
    countries=all_countries_list,
    fill_method="ffill"
)

# Get the raw daily returns for the price matrix
returns = price_matrix.pct_change().dropna()



### -------------- COINTEGRATION RESIDUALS -------------- ###
# Create an instance of the CointegrationResidualGenerator
residual_generator = CointegrationResidualGenerator(price_matrix)

residual_generator.compute_all_asset_residuals()

# Get residuals
asset_residuals = residual_generator.get_asset_residuals()

# Get the input for CNN
# cnn_input contains a set of 329 data samples, each sample represents 30-day cumulative residuals for the 31 countries
cumulative_residual_window = 30
cnn_input = residual_generator.prepare_cnn_input_from_residuals(window=cumulative_residual_window)

# Get the start index of the first 30-day cumulative residuals in the returns DataFrame
start_idx_in_returns = returns.index.get_loc(asset_residuals.index[0])
num_samples = len(asset_residuals) - cumulative_residual_window + 1
next_day_indices = [start_idx_in_returns + i + cumulative_residual_window for i in range(num_samples)]

# Get the next-day returns for the corresponding indices
# The next-day returns are the returns for the day after the last day of each 30-day window
# For example, if the first 30-day window ends on index 0, the next day return is at index 1
# If the second 30-day window ends on index 1, the next day return is at index 2, and so on.
next_day_returns = returns.iloc[next_day_indices]




### -------------- FEED THE 30-DAY CUMULATIVE RESIDUALS OF EVERY COUNTRY TO CNN+FFN -------------- ###
# Transform cnn_input to be compatible with the CNN input shape
cnn_input_array = cnn_input.transpose(0, 2, 1) # [samples, features, window]

# FILIP'S SECTION: CNN+FFN
# Hey Filip, this is the section where you can add your code to train the CNN+FFN model.
# cnn_input_array essentially contains 329 training data points, each data point is 30-day cumulative residuals for the 31 countries.
# So, for one "data point" you would feed the set of 30-day cumulative residuals for every country to the CNN+FFN model.
# One data point should result in a set of weights for each of the 31 countries.
# Each set of weights is used to calculate one next-day portfolio return.
# Repeat this for all 329 data points to get a set of 329 portfolio returns.
# You can then use these portfolio returns to calculate the Sharpe ratio
# Optimize the CNN+FFN to maximize the Sharpe ratio.

# Set the device to GPU if available, otherwise MPS (for Mac silicon) or CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

torch.manual_seed(1)  # For reproducibility

# Create the PortfolioOptimizer instance and train the model
# NOTE: We need to have some validation data to evaluate the model performance
# otherwise we will be overfitting the model to the training data

# Grid search for hyperparameters
num_filters = [8, 16, 32]
filter_sizes = [3, 5, 7]
hidden_dims = [64, 128]

# Choose the best hyperparameters based on validation performance
best_score = float('-inf')
best_params = None

# Training and validation split
# 70% training, 15% validation, 15% testing
# Note: The test set is not used in the training process, but it can be used ONCE to evaluate the final model performance
# The validation set is used to tune the hyperparameters and avoid overfitting
"""
train_dataset = cnn_input_array[:int(0.7 * len(cnn_input_array))]
train_next_day_returns = next_day_returns[:int(0.7 * len(next_day_returns))]

val_dataset = cnn_input_array[int(0.7 * len(cnn_input_array)):]
val_next_day_returns = next_day_returns[int(0.7 * len(next_day_returns)):]

test_dataset = val_dataset[int(0.5 * len(val_dataset)):]
test_next_day_returns = val_next_day_returns[int(0.5 * len(val_next_day_returns)):]
"""
datasetSplitter = TimeSeriesDataSplitter(data_x=cnn_input_array, data_y=next_day_returns, train_size=0.7, validation_size=0.15, test_size=0.15)
(train), (val), (test) = datasetSplitter.split_data()

# Unpack the datasets
train_dataset, train_next_day_returns = train
val_dataset, val_next_day_returns = val
test_dataset, test_next_day_returns = test

print(f"Train dataset shape: {train_dataset.shape}, Train next day returns shape: {train_next_day_returns.shape}")
print(f"Validation dataset shape: {val_dataset.shape}, Validation next day returns shape: {val_next_day_returns.shape}")
print(f"Test dataset shape: {test_dataset.shape}, Test next day returns shape: {test_next_day_returns.shape}")

for num_filter in num_filters:
    for filter_size in filter_sizes:
        for hidden_dim in hidden_dims:
                print(f"Training with filters: {num_filter}, filter size: {filter_size}, hidden dim: {hidden_dim}")
                
                # Initialize the PortfolioOptimizer
                optimizer = PortfolioOptimizer(train_dataset, 
                                               train_next_day_returns, 
                                               val_dataset,
                                               val_next_day_returns,
                                               batch_size=1000, 
                                               num_epochs=1000, 
                                               num_filters=num_filter, 
                                               device=device, 
                                               filter_size=filter_size,
                                               hidden_dim=hidden_dim)
                
                # Train the model
                _, portfolio_returns = optimizer.train(verbose=False) # The final sharpe should be from the validation set

                # Evaluate the model on the validation set
                val_sharpe, _ = optimizer.evaluate_final()
                if val_sharpe > best_score:
                    best_score = val_sharpe
                    best_params = (num_filter, filter_size, hidden_dim)
                    best_portfolio_returns = portfolio_returns
    
print(f"Best parameters: {best_params} with Sharpe Ratio on Validation Set: {best_score}")

### -------------- GET THE WEIGHTS OUTPUTTED FROM CNN+FFN -------------- ###
# Initializing the Sharpe ratio evaluator
#evaluator = BacktestSharpeEvaluator()

# Get the weight outputted from the CNN+FFN model
# One weight for each country
#weights = np.array([]) # CHANGE THIS TO THE ACTUAL WEIGHTS OUTPUTTED FROM THE CNN+FFN MODEL

# Normalize the weights using L1 normalization
# This is done to ensure that the portfolio is dollar-neutral
#normalized_weights = evaluator.normalize_weights_l1(weights)

# Multiply the normalized weights (vector) with the next-day returns (vector) to get the portfolio return
#next_day_portfolio_return = evaluator.compute_portfolio_return(normalized_weights, next_day_returns.iloc[0].values)

# Store the next day portfolio return 
#evaluator.add_return(next_day_portfolio_return)

# Repeat the above step for all 329 data points, adding the portfolio returns to the evaluator for each data point

# Once all portfolio returns are calculated, you can calculate the Sharpe ratio

# Train the model to optimize the Sharpe Ratio


Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv

--- List of Countries ---
['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'Bulgaria', 'Serbia', 'Croatia', 'Montenegro', 'North Macedonia', 'Ireland']


c:\Users\Volter\Documents\GITHUB REPOSITORIES\stat3007-deep-learning-project\src\DataLoader.py:143: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
c:\Users\Volter\anaconda3\envs\DeepLearningSTAT7007\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Train dataset shape: (1506, 31, 30), Train next day returns shape: (1506, 31)
Validation dataset shape: (322, 31, 30), Validation next day returns shape: (322, 31)
Test dataset shape: (324, 31, 30), Test next day returns shape: (324, 31)
Training with filters: 8, filter size: 3, hidden dim: 64

Final Sharpe Ratio on Validation Set: 0.0151
Training with filters: 8, filter size: 3, hidden dim: 128

Final Sharpe Ratio on Validation Set: -0.0133
Training with filters: 8, filter size: 5, hidden dim: 64

Final Sharpe Ratio on Validation Set: 0.0267
Training with filters: 8, filter size: 5, hidden dim: 128

Final Sharpe Ratio on Validation Set: 0.0520
Training with filters: 8, filter size: 7, hidden dim: 64

Final Sharpe Ratio on Validation Set: 0.0761
Training with filters: 8, filter size: 7, hidden dim: 128

Final Sharpe Ratio on Validation Set: 0.0731
Training with filters: 16, filter size: 3, hidden dim: 64

Final Sharpe Ratio on Validation Set: 0.0154
Training with filters: 16, filter si

KeyboardInterrupt: 

## Data Preperation: rolling-window training data (2015-01-01 to 2024-01-01) and rolling-window testing data (2024-01-02 to 2025-01-02)

In [32]:
# Initialize the DataLoader with the test data
parser = DataLoader(file_path="../data/european_wholesale_electricity_price_data_daily.csv")

# Get list of countries
all_countries_list = parser.get_country_list()

# Define the train and test time ranges
train_time_range = "2015-01-01,2024-01-01"
test_time_range = "2024-01-02,2025-01-02"

# Define the window size and stride
one_window_days = 365
window_stride_days = 5

# Get the rolling window of the test data
x_tr_data = parser.get_price_matrix_rolling_window(
    one_window_days=one_window_days,
    window_stride_days=window_stride_days,
    time_range=train_time_range,
    countries=["Norway", "Germany"],
    fill_method="ffill",
)

# Get the rolling window of the test data
x_ts_data = parser.get_price_matrix_rolling_window(
    one_window_days=one_window_days,
    window_stride_days=window_stride_days,
    time_range=test_time_range,
    countries=["Norway", "Germany"],
    fill_method="ffill",
)


# -----------------------------------------------------

# Get the price matrix for the training data
price_matrix_tr = parser.get_price_matrix(
    time_range=train_time_range,
    countries=["Norway", "Germany"],
    fill_method="ffill",
)

# Get the price matrix for the test data
price_matrix_ts = parser.get_price_matrix(
    time_range=test_time_range,
    countries=["Norway", "Germany"],
    fill_method="ffill",
)

# Get the next-day returns for each rolling window in x_tr_data
y_tr_data = parser.get_next_day_returns(
    rolling_windows=x_tr_data,
    price_matrix=price_matrix_tr,
)

# Get the next-day returns for each rolling window in x_ts_data
y_ts_data = parser.get_next_day_returns(
    rolling_windows=x_ts_data,
    price_matrix=price_matrix_ts,
)

# print("------ sanity check ------")
# # Print the contents of the n'th window of x_tr_data along with the corresponding y_tr_data
# n = len(x_tr_data) - 1 # Get the last window for demonstration
# nth_window = x_tr_data[n]
# print("First window of x_tr_data:")
# print(nth_window)
# print("\nCorresponding y_tr_data:")
# print(y_tr_data[n])
# print("\n")

# --------------------------------------------------------
x_tr_data_cumulative_residuals = []
y_tr_data_next_day_returns = []
returns = price_matrix_tr.pct_change().dropna()

# index = 0
# current_price_matrix = x_tr_data[index]
# # Print the shape of the current price matrix
# print(f"Current price matrix shape: {current_price_matrix.shape}")
for current_price_matrix in x_tr_data:


    # Create an instance of the CointegrationResidualGenerator
    residual_generator = CointegrationResidualGenerator(current_price_matrix)

    residual_generator.compute_all_asset_residuals()

    # Get residuals
    asset_residuals = residual_generator.get_asset_residuals()

    # Get the input for CNN
    # cnn_input contains a set of 329 data samples, each sample represents 30-day cumulative residuals for the 31 countries
    cumulative_residual_window = 30
    cnn_input = residual_generator.prepare_cnn_input_from_residuals(window=cumulative_residual_window)

    # Reshape cnn_input to be compatible with the CNN input shape
    cnn_input_array = cnn_input.transpose(0, 2, 1) # [samples, features, window]

    # Get the start index of the first 30-day cumulative residuals in the returns DataFrame
    start_idx_in_returns = returns.index.get_loc(asset_residuals.index[0])
    num_samples = len(asset_residuals) - cumulative_residual_window + 1
    next_day_indices = [start_idx_in_returns + i + cumulative_residual_window for i in range(num_samples)]

    # Get the next-day returns for the corresponding indices
    # The next-day returns are the returns for the day after the last day of each 30-day window
    # For example, if the first 30-day window ends on index 0, the next day return is at index 1
    # If the second 30-day window ends on index 1, the next day return is at index 2, and so on.
    next_day_returns = returns.iloc[next_day_indices]

    # # SANITY CHECK
    # # Extract the last dates of the cumulative residuals from asset_residuals
    # last_dates_of_residuals = asset_residuals.index[-num_samples:]  # Last dates for each cumulative residual window

    # # Extract the dates of next_day_returns
    # next_day_dates = next_day_returns.index

    # # Perform the sanity check
    # for i, (last_date, next_day_date) in enumerate(zip(last_dates_of_residuals, next_day_dates)):
    #     print(f"Sample {i + 1}:")
    #     print(f"  Last date of cumulative residuals: {last_date}")
    #     print(f"  Date of next-day return: {next_day_date}")
    #     if next_day_date == last_date + pd.Timedelta(days=1):
    #         print("  Alignment: Correct")
    #     else:
    #         print("  Alignment: Incorrect")
    
    # Add the cnn_input_array and next_day_returns to the training data lists
    x_tr_data_cumulative_residuals.append(cnn_input_array)
    y_tr_data_next_day_returns.append(next_day_returns.values)

    # print the contents of the first element of x_tr_data_cumulative_residuals and y_tr_data_next_day_returns
    first_element_x = x_tr_data_cumulative_residuals[0]
    first_element_y = y_tr_data_next_day_returns[0]
    


Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv


c:\Users\Volter\Documents\GITHUB REPOSITORIES\stat3007-deep-learning-project\src\DataLoader.py:186: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
c:\Users\Volter\Documents\GITHUB REPOSITORIES\stat3007-deep-learning-project\src\DataLoader.py:186: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
c:\Users\Volter\Documents\GITHUB REPOSITORIES\stat3007-deep-learning-project\src\DataLoader.py:143: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
c:\Users\Volter\Documents\GITHUB REPOSITORIES\stat3007-deep-learning-project\src\DataLoader.py:143: FutureWarning: DataFr